In [1]:
import csv
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import pickle

## Ann

In [2]:
# file = open('processed_features', 'rb')
# X_train, Y_train, X_test, Y_test, X_val, Y_val = pickle.load(file)
# file.close()

file = open('data_pickles/processed_features_plus', 'rb')
X_train, Y_train, X_test, Y_test, X_val, Y_val = pickle.load(file)
file.close()

# file = open('processed_features_window_2', 'rb')
# X_train, Y_train, X_test, Y_test, X_val, Y_val = pickle.load(file)
# file.close()

# file = open('processed_features_window_3', 'rb')
# X_train, Y_train, X_test, Y_test, X_val, Y_val = pickle.load(file)
# file.close()

In [3]:
X_train.shape, X_train[1], Y_train.shape

((870, 13),
 array([0.44308452, 0.34832337, 0.1956427 , 0.53335732, 0.61453386,
        0.18496241, 0.24151003, 0.47051729, 0.63403149, 0.48049792,
        0.5156211 , 0.11363636, 0.5       ]),
 (870,))

In [4]:
reshape_form = X_train.shape[1]

X_train = X_train.reshape(len(X_train), reshape_form)
X_test = X_test.reshape(len(X_test), reshape_form)
X_val = X_val.reshape(len(X_val), reshape_form)

In [5]:
visible = Input(shape=(13, ))
hidden1 = Dense(16, activation='relu')(visible)
# drop1 = Dropout(0.25)(hidden1)
hidden2 = Dense(8, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)
model = Model(inputs=visible, outputs=output)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense (Dense)               (None, 16)                224       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 369
Trainable params: 369
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

callback = EarlyStopping(monitor='accuracy', patience=10)

In [7]:
history = model.fit(X_train, Y_train,
                    # batch_size=100,
                    epochs=50,
                    verbose=2,
                    validation_data=(X_val, Y_val),
                    callbacks=[callback])

Epoch 1/50
28/28 - 1s - loss: 0.6870 - accuracy: 0.5977 - val_loss: 0.6713 - val_accuracy: 0.6452 - 1s/epoch - 43ms/step
Epoch 2/50
28/28 - 0s - loss: 0.6548 - accuracy: 0.7011 - val_loss: 0.6417 - val_accuracy: 0.7151 - 112ms/epoch - 4ms/step
Epoch 3/50
28/28 - 0s - loss: 0.6307 - accuracy: 0.7552 - val_loss: 0.6182 - val_accuracy: 0.7366 - 117ms/epoch - 4ms/step
Epoch 4/50
28/28 - 0s - loss: 0.6091 - accuracy: 0.7586 - val_loss: 0.5954 - val_accuracy: 0.7151 - 109ms/epoch - 4ms/step
Epoch 5/50
28/28 - 0s - loss: 0.5867 - accuracy: 0.7805 - val_loss: 0.5726 - val_accuracy: 0.7849 - 113ms/epoch - 4ms/step
Epoch 6/50
28/28 - 0s - loss: 0.5647 - accuracy: 0.7908 - val_loss: 0.5513 - val_accuracy: 0.7527 - 102ms/epoch - 4ms/step
Epoch 7/50
28/28 - 0s - loss: 0.5414 - accuracy: 0.8034 - val_loss: 0.5261 - val_accuracy: 0.7849 - 112ms/epoch - 4ms/step
Epoch 8/50
28/28 - 0s - loss: 0.5202 - accuracy: 0.8046 - val_loss: 0.5051 - val_accuracy: 0.7796 - 112ms/epoch - 4ms/step
Epoch 9/50
28/28 -

In [8]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Evaluation loss:', score[0])
print('Evaluation accuracy:', score[1])

Evaluation loss: 0.3364165723323822
Evaluation accuracy: 0.8526315689086914
